In [1]:
import numpy as np
from tqdm import tqdm
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import matplotlib.pyplot as plt
import collections 

In [2]:
import argparse
import errno
import os
import random
from importlib.metadata import requires
from timeit import timeit
import dill as pickle
import numpy as np
import scipy
import torch
import wandb
import yaml
from sympy import Matrix, MatrixSymbol, derive_by_array, symarray
from torch.distributions import Categorical

from subrl.utils.environment import GridWorld
from subrl.utils.network import append_state
from subrl.utils.network import policy as agent_net
from subrl.utils.visualization import Visu

In [3]:
workspace = "subrl"

params = {
    "env": {
        "start": 1,
        "step_size": 0.1,
        "shape": {"x": 7, "y": 14},
        "horizon": 40,
        "node_weight": "constant",
        "disc_size": "small",
        "n_players": 3,
        "Cx_lengthscale": 2,
        "Cx_noise": 0.001,
        "Fx_lengthscale": 1,
        "Fx_noise": 0.001,
        "Cx_beta": 1.5,
        "Fx_beta": 1.5,
        "generate": False,
        "env_file_name": 'env_data.pkl',
        "cov_module": 'Matern',
        "stochasticity": 0.0,
        "domains": "two_room",
        "num": 1  # 替代原来的args.env
    },
    "alg": {
        "gamma": 1,
        "type": "NM",
        "ent_coef": 0.0,
        "epochs": 140,
        "lr": 0.02
    },
    "common": {
        "a": 1,
        "subgrad": "greedy",
        "grad": "pytorch",
        "algo": "both",
        "init": "deterministic",
        "batch_size": 3000
    },
    "visu": {
        "wb": "disabled",
        "a": 1
    }
}

print(params)

# 2) Set the path and copy params from file
env_load_path = workspace + \
    "/environments/" + params["env"]["node_weight"]+ "/env_" + \
    str(params["env"]["num"])



epochs = params["alg"]["epochs"]

H = params["env"]["horizon"]
MAX_Ret = 2*(H+1)
if params["env"]["disc_size"] == "large":
    MAX_Ret = 3*(H+2)

# 3) Setup the environement
env = GridWorld(
    env_params=params["env"], common_params=params["common"], visu_params=params["visu"], env_file_path=env_load_path)
node_size = params["env"]["shape"]['x']*params["env"]["shape"]['y']
# TransitionMatrix = torch.zeros(node_size, node_size)

if params["env"]["node_weight"] == "entropy" or params["env"]["node_weight"] == "steiner_covering" or params["env"]["node_weight"] == "GP": 
    a_file = open(env_load_path +".pkl", "rb")
    data = pickle.load(a_file)
    a_file.close()

if params["env"]["node_weight"] == "entropy":
    env.cov = data
if params["env"]["node_weight"] == "steiner_covering":
    env.items_loc = data
if params["env"]["node_weight"] == "GP":
    env.weight = data

visu = Visu(env_params=params["env"])

env.get_horizon_transition_matrix()


{'env': {'start': 1, 'step_size': 0.1, 'shape': {'x': 7, 'y': 14}, 'horizon': 40, 'node_weight': 'constant', 'disc_size': 'small', 'n_players': 3, 'Cx_lengthscale': 2, 'Cx_noise': 0.001, 'Fx_lengthscale': 1, 'Fx_noise': 0.001, 'Cx_beta': 1.5, 'Fx_beta': 1.5, 'generate': False, 'env_file_name': 'env_data.pkl', 'cov_module': 'Matern', 'stochasticity': 0.0, 'domains': 'two_room', 'num': 1}, 'alg': {'gamma': 1, 'type': 'NM', 'ent_coef': 0.0, 'epochs': 140, 'lr': 0.02}, 'common': {'a': 1, 'subgrad': 'greedy', 'grad': 'pytorch', 'algo': 'both', 'init': 'deterministic', 'batch_size': 3000}, 'visu': {'wb': 'disabled', 'a': 1}}
x_ticks [-0.5001, -0.4999, 0.4999, 0.5001, 1.4999, 1.5001, 2.4999, 2.5001, 3.4999, 3.5001, 4.4999, 4.5001, 5.4999, 5.5001, 6.4999, 6.5001, 7.4999, 7.5001, 8.4999, 8.5001, 9.4999, 9.5001, 10.4999, 10.5001, 11.4999, 11.5001, 12.4999, 12.5001, 13.4999, 13.5001]
y_ticks [-0.5001, -0.4999, 0.4999, 0.5001, 1.4999, 1.5001, 2.4999, 2.5001, 3.4999, 3.5001, 4.4999, 4.5001, 5.4999,

In [4]:
def sample_excellent_trajectories(filepath="go_explore_archive_spacetime_10m.pkl", 
                                  method='top_n', 
                                  n=10, 
                                  p=0.1, 
                                  threshold=0):
    """
        Load data from the Go-Explore archive and sample high-quality trajectories based on the specified method.

        Args:
            filepath (str): Path to the .pkl archive file.
            method (str): Sampling method. Options are 'top_n', 'top_p', or 'threshold'.
            n (int): Number of trajectories to sample for the 'top_n' method.
            p (float): Percentage of top trajectories to sample for the 'top_p' method (e.g., 0.1 means top 10%).
            threshold (float): Minimum reward threshold for the 'threshold' method.
        
        Returns:
            list: A list of trajectory dictionaries with high rewards, sorted in descending order of reward.
                  Returns an empty list if the file does not exist or the archive is empty.
    """
    # 1. Check if the file exists and load the data
    if not os.path.exists(filepath):
        print(f"Error: Archive file not found '{filepath}'")
        return []
    
    try:
        with open(filepath, "rb") as f:
            archive = pickle.load(f)
        if not archive:
            print("警告：存檔庫為空。")
            return []
    except Exception as e:
        print(f"讀取文件時出錯: {e}")
        return []

    # 2. 提取所有軌跡數據並按獎勵排序
    # archive.values() 返回的是包含 reward, states, actions 等信息的字典
    all_trajectories_data = list(archive.values())
    
    # 按 'reward' 鍵從高到低排序
    all_trajectories_data.sort(key=lambda x: x['reward'], reverse=True)

    # 3. 根據指定方法進行採樣
    sampled_trajectories = []
    if method == 'top_n':
        # 取獎勵最高的前 N 條
        num_to_sample = min(n, len(all_trajectories_data))
        sampled_trajectories = all_trajectories_data[:num_to_sample]
        print(f"方法: Top-N。從 {len(all_trajectories_data)} 條軌跡中篩選出最好的 {len(sampled_trajectories)} 條。")

    elif method == 'top_p':
        # 取獎勵最高的前 P%
        if not (0 < p <= 1):
            print("錯誤：百分比 'p' 必須在 (0, 1] 之間。")
            return []
        num_to_sample = int(len(all_trajectories_data) * p)
        sampled_trajectories = all_trajectories_data[:num_to_sample]
        print(f"方法: Top-P。從 {len(all_trajectories_data)} 條軌跡中篩選出最好的前 {p*100:.1f}% ({len(sampled_trajectories)} 條)。")

    elif method == 'threshold':
        # 取獎勵高於指定門檻的所有軌跡
        sampled_trajectories = [data for data in all_trajectories_data if data['reward'] >= threshold]
        print(f"方法: Threshold。從 {len(all_trajectories_data)} 條軌跡中篩選出 {len(sampled_trajectories)} 條獎勵不低於 {threshold} 的軌跡。")
        
    else:
        print(f"錯誤：未知的採樣方法 '{method}'。請使用 'top_n', 'top_p', 或 'threshold'。")

    return sampled_trajectories


In [61]:
top_20_trajectories = sample_excellent_trajectories(method='top_n', n=50)
top_20_trajectories_2=sample_excellent_trajectories(
    "go_explore_archive_spacetime_.pkl",method='top_n', n=50)
top_20_trajectories= top_20_trajectories + top_20_trajectories_2
if top_20_trajectories:
    print(f"其中最好的一條獎勵為: {top_20_trajectories[0]['reward']}")
    print(f"最差的一條（在這20條中）獎勵為: {top_20_trajectories[-1]['reward']}\n")

方法: Top-N。從 2312 條軌跡中篩選出最好的 50 條。
方法: Top-N。從 2312 條軌跡中篩選出最好的 50 條。
其中最好的一條獎勵為: 68
最差的一條（在這20條中）獎勵為: 66



In [62]:
# top_20_trajectories = sample_excellent_trajectories(method='top_n', n=1)
# if top_20_trajectories:
#     print(f"其中最好的一條獎勵為: {top_20_trajectories[0]['reward']}")
#     print(f"最差的一條（在這20條中）獎勵為: {top_20_trajectories[-1]['reward']}\n")

In [63]:
def expand_trajectory_states(trajectory_states, H):
    """
    将轨迹状态按照 append_state 的方式进行拓展
    
    Args:
        trajectory_states: 轨迹中的状态列表
        H: 时间范围参数
        
    Returns:
        expanded_states: 拓展后的状态列表
    """
    expanded_states = []
    
    # 模拟原始代码中的 mat_state 构建过程
    mat_state = []
    
    for i, state in enumerate(trajectory_states):
        mat_state.append(state)
        
        # 对于除了最后一个状态外的所有状态，都进行 append_state 拓展
        if i < H - 1:
            # 使用 append_state 函数进行状态拓展
            batch_state = append_state(mat_state, H-1)
            expanded_states.append(batch_state)
        else:
            expanded_states.append(expanded_states[-1])  # 最后一个状态不需要拓展，直接重复最后一个状态
    
    return expanded_states

# 使用示例：拓展最佳轨迹的状态
H = params["env"]["horizon"]  # 使用环境参数中的 horizon
trajectory_states=top_20_trajectories[0]['states']
expanded_trajectory_states = expand_trajectory_states(trajectory_states, H)

print(f"原始轨迹状态数量: {len(trajectory_states)}")
print(f"拓展后状态数量: {len(expanded_trajectory_states)}")

# 查看拓展后的第一个状态的形状
if expanded_trajectory_states:
    print(f"拓展后第一个状态的形状: {expanded_trajectory_states[0].shape}")
expanded_trajectory_states[-2],expanded_trajectory_states[-1]

原始轨迹状态数量: 40
拓展后状态数量: 40
拓展后第一个状态的形状: torch.Size([1, 39])


(tensor([[34., 35., 36., 37., 51., 52., 66., 80., 81., 82., 68., 54., 40., 26.,
          12., 11., 10., 24., 38., 37., 36., 35., 34., 33., 32., 31., 30., 16.,
           2.,  1.,  0., 14., 28., 42., 56., 70., 71., 72., 58.]]),
 tensor([[34., 35., 36., 37., 51., 52., 66., 80., 81., 82., 68., 54., 40., 26.,
          12., 11., 10., 24., 38., 37., 36., 35., 34., 33., 32., 31., 30., 16.,
           2.,  1.,  0., 14., 28., 42., 56., 70., 71., 72., 58.]]))

In [64]:
expert_s = np.array(expanded_trajectory_states[:39]).squeeze()
expert_a = np.array(top_20_trajectories[0]['actions'])
len(expert_s), len(expert_a)

(39, 39)

In [65]:
expert_s=[]
expert_a=[]
for traj_i in top_20_trajectories:
    expert_s.extend(np.array(expand_trajectory_states(traj_i['states'],H)[:-1]).squeeze())
    expert_a.extend(np.array(traj_i['actions']))
expert_s = np.array(expert_s)
expert_a = np.array(expert_a)
len(expert_s), len(expert_a)

(3856, 3856)

In [124]:
def compute_advantage(gamma, lmbda, td_delta, dones=None):
    td_delta = td_delta.detach().numpy()
    advantage_list = []
    advantage = 0.0
    for i, delta in enumerate(td_delta[::-1]):
        if dones is not None:
            done = dones[len(td_delta) - 1 - i]
            advantage = gamma * lmbda * advantage * (1-done) + delta
        else:
            advantage = gamma * lmbda * advantage + delta
        advantage_list.append(advantage)
    advantage_list.reverse()
    return torch.tensor(advantage_list, dtype=torch.float)

In [141]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)


class ValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)


class PPO:
    ''' PPO算法,采用截断方式 '''
    def __init__(self, state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                 lmbda, epochs, eps, gamma, device):
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim).to(device)
        self.critic = ValueNet(state_dim, hidden_dim).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(),
                                                lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),
                                                 lr=critic_lr)
        self.gamma = gamma
        self.lmbda = lmbda
        self.epochs = epochs  # 一条序列的数据用于训练轮数
        self.eps = eps  # PPO中截断范围的参数
        self.device = device

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        probs = self.actor(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action.item()

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(
            self.device)
        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)
        td_target = rewards + self.gamma * self.critic(next_states) * (1 -
                                                                       dones)
        td_delta = td_target - self.critic(states)
        advantage = compute_advantage(self.gamma, self.lmbda,
                                               td_delta.cpu(),dones.cpu()).to(self.device)
        old_log_probs = torch.log(self.actor(states).gather(1,
                                                            actions)+1e-10).detach()

        for _ in range(self.epochs):
            log_probs = torch.log(self.actor(states).gather(1, actions)+1e-10)
            ratio = torch.exp(log_probs - old_log_probs)
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1 - self.eps,
                                1 + self.eps) * advantage  # 截断
            actor_loss = torch.mean(-torch.min(surr1, surr2))  # PPO损失函数
            critic_loss = torch.mean(
                F.mse_loss(self.critic(states), td_target.detach()))
            self.actor_optimizer.zero_grad()
            self.critic_optimizer.zero_grad()
            actor_loss.backward()
            critic_loss.backward()
            self.actor_optimizer.step()
            self.critic_optimizer.step()

In [142]:
class Discriminator(nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Discriminator, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x, a):
        cat = torch.cat([x, a], dim=1)
        x = F.relu(self.fc1(cat))
        return torch.sigmoid(self.fc2(x))
class GAIL:
    def __init__(self, agent, state_dim, action_dim, hidden_dim, lr_d):
        self.discriminator = Discriminator(state_dim, hidden_dim,
                                           action_dim).to(device)
        self.discriminator_optimizer = torch.optim.Adam(
            self.discriminator.parameters(), lr=lr_d)
        self.agent = agent

    def learn(self, expert_s, expert_a, agent_s, agent_a, next_s, dones):
        expert_states = torch.tensor(expert_s, dtype=torch.float).to(device)
        expert_actions = torch.tensor(expert_a).to(device)
        agent_states = torch.tensor(agent_s, dtype=torch.float).to(device)
        agent_actions = torch.tensor(agent_a).to(device)
        expert_actions = F.one_hot(expert_actions, num_classes=5).float()
        agent_actions = F.one_hot(agent_actions, num_classes=5).float()

        expert_prob = self.discriminator(expert_states, expert_actions)
        agent_prob = self.discriminator(agent_states, agent_actions)
        discriminator_loss = nn.BCELoss()(
            agent_prob, torch.ones_like(agent_prob)) + nn.BCELoss()(
                expert_prob, torch.zeros_like(expert_prob))
        self.discriminator_optimizer.zero_grad()
        discriminator_loss.backward()
        self.discriminator_optimizer.step()

        rewards = -torch.log(agent_prob).detach().cpu().numpy()
        transition_dict = {
            'states': agent_s,
            'actions': agent_a,
            'rewards': rewards,
            'next_states': next_s,
            'dones': dones
        }
        self.agent.update(transition_dict)

In [143]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def train():
    params["common"]["batch_size"]=100      #采样的batch大小
    actor_lr = 1e-3
    critic_lr = 1e-2
    num_episodes = 250
    hidden_dim = 128
    gamma = 0.98
    lmbda = 0.95
    epochs = 10
    eps = 0.2
    state_dim = H-1
    action_dim = 5
    torch.manual_seed(0)
    lr_d = 1e-3
    agent = PPO(state_dim, hidden_dim, action_dim, actor_lr, critic_lr, lmbda,
                epochs, eps, gamma, device)
    gail = GAIL(agent, state_dim, action_dim, hidden_dim, lr_d)
    n_episode = 2000
    return_list = []

    with tqdm(total=n_episode, desc="进度条") as pbar:
        for i in range(n_episode):
            episode_return = 0
            mat_state = []
            mat_return = []
            env.initialize()
            mat_state.append(env.state)
            state_list = []
            action_list = []
            next_state_list = []
            done_list = []
            state_lists = [[] for _ in range(params["common"]["batch_size"])]
            action_lists = [[] for _ in range(params["common"]["batch_size"])]
            next_state_lists = [[] for _ in range(params["common"]["batch_size"])]
            done_lists = [[] for _ in range(params["common"]["batch_size"])]
            for h_iter in range(H-1):
                batch_state = append_state(mat_state, H-1)
                probs = agent.actor(batch_state.to(device))
                actions_dist = torch.distributions.Categorical(probs)
                actions = actions_dist.sample()
                env.step(h_iter, actions.cpu())
                mat_state.append(env.state)  # s+1
                mat_return.append(env.weighted_traj_return(mat_state, type = params["alg"]["type"]))
                if h_iter == 0:
                    reward = mat_return[-1]
                else:
                    reward = mat_return[-1]-mat_return[-2]

                if h_iter == H-2:
                    next_state = batch_state
                    done = 1
                else:
                    next_state = append_state(mat_state, H-1)
                    done = 0

                for j in range(params["common"]["batch_size"]):
                    state_lists[j].append(np.array(batch_state[j]))
                    action_lists[j].append(actions[j])
                    next_state_lists[j].append(np.array(next_state[j]))
                    done_lists[j].append(done)
            for j in range(params["common"]["batch_size"]):
                state_list.extend(state_lists[j])
                action_list.extend(action_lists[j])
                next_state_list.extend(next_state_lists[j])
                done_list.extend(done_lists[j])
            return_list.append(mat_return[-1].float().mean())
            gail.learn(expert_s, expert_a, state_list, action_list,
                    next_state_list, done_list)
            if (i + 1) % 10 == 0:
                pbar.set_postfix({'return': '%.3f' % np.mean(return_list[-10:])})
            pbar.update(1)
    return agent

In [144]:
min_return = []
max_return = []
mean_return = []
median_return = []
for iter in range(10):
    agent = train()
    params["common"]["batch_size"]=3000
    mat_state = []
    mat_return = []
    env.initialize()
    mat_state.append(env.state)
    init_state = env.state
    for h_iter in range(H-1):
        batch_state = append_state(mat_state, H-1)
        probs = agent.actor(batch_state.to(device))
        actions_dist = torch.distributions.Categorical(probs)
        actions = actions_dist.sample().cpu()
        env.step(h_iter, actions)
        mat_state.append(env.state)  # s+1

    returns = env.weighted_traj_return(mat_state, type = params["alg"]["type"]).float()
    min_return.append(returns.min())
    max_return.append(returns.max())
    mean_return.append(returns.mean())
    median_return.append(returns.median())
mean_min_return = np.mean(min_return)
std_min_return = np.std(min_return)
mean_max_return = np.mean(max_return)
std_max_return = np.std(max_return)
mean_mean_return = np.mean(mean_return)
std_mean_return = np.std(mean_return)
mean_median_return = np.mean(median_return)
std_median_return = np.std(median_return)
print(f"min: {mean_min_return:.2f}±{std_min_return:.2f}, max: {mean_max_return:.2f}±{std_max_return:.2f}, mean: {mean_mean_return:.2f}±{std_mean_return:.2f}, median: {mean_median_return:.2f}±{std_median_return:.2f}")

进度条:   0%|          | 0/2000 [00:00<?, ?it/s]C:\Users\ZHY\AppData\Local\Temp\ipykernel_19396\613001409.py:58: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  state_lists[j].append(np.array(batch_state[j]))
C:\Users\ZHY\AppData\Local\Temp\ipykernel_19396\613001409.py:60: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  next_state_lists[j].append(np.array(next_state[j]))
C:\Users\ZHY\AppData\Local\Temp\ipykernel_19396\310866930.py:8: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  advantage = gamma * lmbda * advantage * (1-done) + delta
进度条:   9%|▉         | 181/2000 [03:37<36:25,  1.20s/it, return=19.314]


KeyboardInterrupt: 

In [116]:
device = torch.device("cpu")
torch.manual_seed(0)
def train():
    params["common"]["batch_size"]=100
    # Agent's policy
    if params["alg"]["type"]=="M" or params["alg"]["type"]=="SRL":
        agent = agent_net(2, env.action_dim)
    else:
        agent = agent_net(H-1, env.action_dim)

    state_dim = H-1
    action_dim = 5
    hidden_dim = 128
    lr_d = 1e-3
    optim = torch.optim.Adam(agent.parameters(), lr=params["alg"]["lr"])
    discriminator = Discriminator(state_dim, hidden_dim,action_dim).to(device)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr_d)

    for t_eps in range(epochs):
        mat_action = []
        mat_state = []
        mat_return = []
        marginal_return = []
        mat_done = []
        # print(t_eps)
        env.initialize()
        mat_state.append(env.state)
        init_state = env.state
        list_batch_state = []
        for h_iter in range(H-1):
            if params["alg"]["type"]=="M" or params["alg"]["type"]=="SRL":
                batch_state = mat_state[-1].reshape(-1, 1).float()
                # append time index to the state
                batch_state = torch.cat(
                    [batch_state, h_iter*torch.ones_like(batch_state)], 1)
            else:
                batch_state = append_state(mat_state, H-1)
            action_prob = agent(batch_state)
            policy_dist = Categorical(action_prob)
            actions = policy_dist.sample()
            mat_action.append(actions)
            env.step(h_iter, actions)
            mat_state.append(env.state)  # s+1
            mat_return.append(env.weighted_traj_return(mat_state, type = params["alg"]["type"]))
            if h_iter ==0:
                marginal_return.append(mat_return[h_iter])
            else:
                marginal_return.append(mat_return[h_iter] - mat_return[h_iter-1])
            list_batch_state.append(batch_state)

        ###################
        # Compute gradients
        ###################

        states_visited = torch.vstack(list_batch_state).float()
    
        policy_dist = Categorical(agent(states_visited))
        log_prob = policy_dist.log_prob(torch.hstack(mat_action).to(device))
        # batch_return = torch.hstack(marginal_return)/MAX_Ret
        # print("batch_return", batch_return.shape)

        expert_states = torch.tensor(expert_s, dtype=torch.float).to(device)
        expert_actions = torch.tensor(expert_a).to(device)
        agent_states = states_visited.to(device)
        agent_actions = torch.hstack(mat_action).to(device)
        # print(agent_states.shape, agent_actions.shape, expert_states.shape, expert_actions.shape)
        expert_actions = F.one_hot(expert_actions, num_classes=5).float()
        agent_actions = F.one_hot(agent_actions, num_classes=5).float()

        expert_prob = discriminator(expert_states, expert_actions)
        agent_prob = discriminator(agent_states, agent_actions)
        discriminator_loss = nn.BCELoss()(
            agent_prob, torch.ones_like(agent_prob)) + nn.BCELoss()(
                expert_prob, torch.zeros_like(expert_prob))
        discriminator_optimizer.zero_grad()
        discriminator_loss.backward()
        discriminator_optimizer.step()

        rewards = -torch.log(agent_prob).detach()
        batch_return = rewards.squeeze()
        # print("batch_return", batch_return.shape)
    
        # - 2*policy_dist.entropy().mean()
        J_obj = -1*(torch.mean(log_prob*batch_return) + params["alg"]["ent_coef"] *
                    policy_dist.entropy().mean()/(t_eps+1))
        optim.zero_grad()
        J_obj.backward()
        optim.step()

        obj = env.weighted_traj_return(mat_state).float()
        print(visu.JPi_optimal, " mean ", obj.mean(), " max ",
            obj.max(), " median ", obj.median(), " min ", obj.min(), " ent ", policy_dist.entropy().mean().detach())
    return agent

In [117]:
min_return = []
max_return = []
mean_return = []
median_return = []
for iter in range(10):
    agent = train()
    params["common"]["batch_size"]=3000
    mat_state = []
    mat_return = []
    env.initialize()
    mat_state.append(env.state)
    init_state = env.state
    for h_iter in range(H-1):
        batch_state = append_state(mat_state, H-1)
        probs = agent.actor(batch_state.to(device))
        actions_dist = torch.distributions.Categorical(probs)
        actions = actions_dist.sample().cpu()
        env.step(h_iter, actions)
        mat_state.append(env.state)  # s+1

    returns = env.weighted_traj_return(mat_state, type = params["alg"]["type"]).float()
    min_return.append(returns.min())
    max_return.append(returns.max())
    mean_return.append(returns.mean())
    median_return.append(returns.median())
mean_min_return = np.mean(min_return)
std_min_return = np.std(min_return)
mean_max_return = np.mean(max_return)
std_max_return = np.std(max_return)
mean_mean_return = np.mean(mean_return)
std_mean_return = np.std(mean_return)
mean_median_return = np.mean(median_return)
std_median_return = np.std(median_return)
print(f"min: {mean_min_return:.2f}±{std_min_return:.2f}, max: {mean_max_return:.2f}±{std_max_return:.2f}, mean: {mean_mean_return:.2f}±{std_mean_return:.2f}, median: {mean_median_return:.2f}±{std_median_return:.2f}")

None  mean  tensor(17.0400)  max  tensor(32.)  median  tensor(17.)  min  tensor(8.)  ent  tensor(1.5895)
None  mean  tensor(22.2100)  max  tensor(30.)  median  tensor(22.)  min  tensor(12.)  ent  tensor(1.4336)
None  mean  tensor(21.6300)  max  tensor(33.)  median  tensor(22.)  min  tensor(14.)  ent  tensor(1.2499)
None  mean  tensor(20.3300)  max  tensor(29.)  median  tensor(20.)  min  tensor(11.)  ent  tensor(1.1670)
None  mean  tensor(20.1900)  max  tensor(31.)  median  tensor(20.)  min  tensor(11.)  ent  tensor(1.0304)
None  mean  tensor(19.6800)  max  tensor(32.)  median  tensor(19.)  min  tensor(11.)  ent  tensor(0.9352)
None  mean  tensor(19.8100)  max  tensor(29.)  median  tensor(19.)  min  tensor(11.)  ent  tensor(0.8841)
None  mean  tensor(19.4200)  max  tensor(29.)  median  tensor(19.)  min  tensor(13.)  ent  tensor(0.8449)
None  mean  tensor(18.3200)  max  tensor(29.)  median  tensor(18.)  min  tensor(11.)  ent  tensor(0.8211)
None  mean  tensor(19.2800)  max  tensor(30.)  

In [ ]:
iteration_list = list(range(len(return_list)))
plt.plot(iteration_list, return_list)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('GAIL')
plt.show()

NameError: name 'return_list' is not defined